# 1. Business Understanding & Data Collection

In [1]:
#data
import yfinance as yf

#mapulating data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as data
import seaborn as sns
import math

#model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, classification_report, mean_absolute_percentage_error, mean_absolute_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBRegressor

In [2]:
# Các hàm hỗ trợ
def moving_average(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window=window).mean()

def exponential_moving_average(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def obv(df: pd.DataFrame) -> pd.Series:
    direction = np.sign(df['close'].diff()).fillna(0)
    return (direction * df['volume']).cumsum()

def force_index(df: pd.DataFrame, span: int = 1) -> pd.Series:
    fi = (df['close'] - df['close'].shift(1)) * df['volume']
    return fi.ewm(span=span, adjust=False).mean() if span > 1 else fi

def vroc(series: pd.Series, window: int = 5) -> pd.Series:
    prev = series.shift(window)
    return (series - prev) / prev.replace(0, np.nan)

def range_and_return(df: pd.DataFrame) -> pd.DataFrame:
    price_range = df['high'] - df['low']
    daily_return = df['close'].pct_change()
    return pd.DataFrame({'Range': price_range, 'Return': daily_return})

def volume_momentum(series: pd.Series, window: int = 5) -> pd.Series:
    return series - series.shift(window)

#Áp dụng các chỉ báo vào DataFrame
def custom_feature_func(df: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    """
    Tạo thêm các feature từ volume, giá và thời gian.
    """
    # 1. Feature giá
    df['date'] = pd.to_datetime(df['date'])
    df['Close_Lag1'] = df['close'].shift(1)
    df['Close_Change'] = df['close'].pct_change()
    df['Close_MA5'] = moving_average(df['close'], 5)

    df['Candle_Body'] = (df['close'] - df['open']).abs()
    df['Upper_Shadow'] = df['high'] - df[['close', 'open']].max(axis=1)
    df['Lower_Shadow'] = df[['close', 'open']].min(axis=1) - df['low']

    # 2. Feature volume cơ bản
    df['Volume_MA'] = moving_average(df['volume'], window)
    df['Volume_EMA'] = exponential_moving_average(df['volume'], window)
    df['Volume_STD'] = df['volume'].rolling(window=window).std()
    df['Volume_Momentum'] = volume_momentum(df['volume'], window)
    df['VROC'] = vroc(df['volume'], window=window)

    # 3. Chỉ báo khác từ volume và giá
    df['OBV'] = obv(df)
    df['Force_Index'] = force_index(df, span=window)

    rr = range_and_return(df)
    df['Range'] = rr['Range']
    df['Return'] = rr['Return']

    # 4. Feature lag và rolling trên volume
    df['Volume_Lag1'] = df['volume'].shift(1)
    df['Volume_Rolling_Max_30'] = df['volume'].rolling(window=30).max()
    df['Volume_Rolling_Mean_30'] = df['volume'].rolling(window=30).mean()
    df['Volume_Zscore_30'] = (
        (df['volume'] - df['Volume_Rolling_Mean_30']) /
        df['volume'].rolling(window=30).std()
    )
    df['Volume_Percentile_30'] = df['volume'].rolling(window=30).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1], raw=False
    )# 5. Feature thời gian
    df['Day_of_Week'] = df['date'].dt.dayofweek
    df['Is_Month_End'] = df['date'].dt.is_month_end.astype(int)
    df['Is_Month_Start'] = df['date'].dt.is_month_start.astype(int)

    return df


def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    - Gọi custom_feature_func để tạo toàn bộ feature.
    - Loại bỏ cột giá gốc, giữ lại date, volume và các feature mới.
    - Xóa NaN và sắp xếp theo date.
    """
    df = df.copy()
    df = custom_feature_func(df)
    df.drop(['open', 'high', 'low', 'close'], axis=1, inplace=True, errors='ignore')
    df.dropna(inplace=True)
    df = df.sort_values(by='date', ascending=True).reset_index(drop=True)
    return df

In [ ]:

#boxplot
def boxplot_features(df):
  n_cols = df.shape[1]
  n_rows = math.ceil(n_cols / 5)

  fig, axes = plt.subplots(n_rows, 5, figsize=(12, 5 * n_rows))
  axes = axes.flatten()

  for i, col in enumerate(df.columns):
      sns.boxplot(y=df[col], ax=axes[i])
      axes[i].set_title(col)
  for j in range(i + 1, len(axes)):
      axes[j].axis('off')

  plt.tight_layout()
  plt.show()

#Histogram
def hist_features(df):
  n_cols = df.shape[1]
  n_rows = math.ceil(n_cols / 5)

  fig, axes = plt.subplots(n_rows, 5, figsize=(12, 5 * n_rows))
  axes = axes.flatten()

  for i, col in enumerate(df.columns):
      sns.histplot(df[col], ax=axes[i])
      axes[i].set_title(col)
  for j in range(i + 1, len(axes)):
      axes[j].axis('off')

  plt.tight_layout()
  plt.show()

#Line chart

def line_features(df):
    df = df.sort_values('date')
    n_cols = df.shape[1] - 1
    n_rows = math.ceil(n_cols / 5)

    fig, axes = plt.subplots(n_rows, 5, figsize=(25, 5 * n_rows))
    axes = axes.flatten()

    feature_cols = [col for col in df.columns if col != 'date']

    for i, col in enumerate(feature_cols):
        axes[i].plot(df['date'], df[col])
        axes[i].set_title(col)
        axes[i].tick_params(axis='x', rotation=45)

    # Ẩn subplot thừa
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
df = yf.download("GC=F", start="2015-1-2", end="2025-5-25")

df = df.reset_index()
df.columns = ['date', 'close', 'high',	'low',	'open',	'volume']
df = df.sort_values(by='date', ascending=True)

# 2. Data Description & Preprocessing

In [ ]:
#df trước xử lý
display(df.head())
df.info()

In [ ]:
round(df.drop(columns=['date'], axis=1).describe(),3)

In [ ]:
#df sau khi xử lý
df_model = preprocess_data(df)
display(df_model.info())
df_model.head()

In [ ]:
#Boxplot
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(18, 12))
axes = axes.flatten()

for i in range(24):
    axes[i].boxplot(df_model.drop(['date'],axis=1).iloc[:, i].dropna())
    axes[i].set_title(df_model.drop(['date'],axis=1).columns[i])
    axes[i].tick_params(axis='x', labelrotation=45)

plt.tight_layout()
plt.show()

# 3. Data Analysis (Python)

In [ ]:
# Thống kê mô tả
def describe_data(df):
    print("== Thông tin DataFrame ==")
    print(df.info())
    print("\n== Thống kê mô tả ==")
    display(df.describe().style.background_gradient(cmap='Blues'))
    print("\n== Số lượng giá trị bị thiếu ==")
    print(df.isnull().sum())
describe_data(df)

In [ ]:
# Correlation
# Chọn các cột cần phân tích
cols = ['close', 'open', 'high', 'low', 'volume']
correlation_matrix = df[cols].corr(method='pearson')

# Làm tròn 2 chữ số
correlation_matrix_rounded = correlation_matrix.round(2)

# In ra bảng tương quan
print(correlation_matrix_rounded)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Heatmap tương quan giữa các biến")
plt.show()

In [ ]:
# trend bằng đường trung bình động (Moving Average)
plt.figure(figsize=(14, 6))
plt.plot(df['date'], df['volume'], label='Volume', color='gray', alpha=0.4)
plt.plot(df['date'], df['volume'].rolling(window=20).mean(), label='MA20 Volume', color='red')
plt.plot(df['date'], df['volume'].rolling(window=50).mean(), label='MA50 Volume', color='green')
plt.title("Xu hướng khối lượng giao dịch với MA20 & MA50")
plt.xlabel("Ngày")
plt.ylabel("Khối lượng giao dịch")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 4. Data Visualization (Advanced)

In [ ]:
# 1. Histogram - Phân phối volume
plt.figure(figsize=(8,4))
sns.histplot(df_model['volume'], bins=30, kde=True, color='skyblue')
plt.title("Phân phối Volume")
plt.xlabel("Volume")
plt.ylabel("Tần suất")
plt.show()

In [ ]:
# 2. Line Plot - Biểu đồ Volume theo thời gian
plt.figure(figsize=(12,5))
plt.plot(df_model['date'], df_model['volume'], color='green')
plt.title("Volume theo thời gian")
plt.xlabel("Ngày")
plt.ylabel("Volume")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 3. Bar Chart - Trung bình volume theo ngày trong tuần
day_names = ['Thứ 2', 'Thứ 3', 'Thứ 4', 'Thứ 5', 'Thứ 6', 'Thứ 7', 'Chủ nhật']
df_model['Day_of_Week_Label'] = df_model['Day_of_Week'].map(dict(enumerate(day_names)))
avg_volume = df_model.groupby('Day_of_Week_Label')['volume'].mean()

plt.figure(figsize=(8,4))
avg_volume.plot(kind='bar', color='orange')
plt.title('Volume trung bình theo thứ trong tuần')
plt.xlabel('Day of Week')
plt.ylabel('Average Volume')
plt.show()

In [ ]:
# 4. Scatter Plot - Volume vs % thay đổi giá
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_model, x='Close_Change', y='volume', alpha=0.5)
plt.title('Volume vs % thay đổi giá')
plt.xlabel('% Thay đổi giá (Close_Change)')
plt.ylabel('Volume giao dịch')
plt.grid(True)
plt.show()

In [ ]:
# 5. Top 10 ngày có volume cao nhất
top10 = df_model.sort_values('volume', ascending=False).head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top10['date'].dt.strftime('%Y-%m-%d'),
            y=top10['volume'],
            color='skyblue')
plt.title("Top 10 ngày có volume cao nhất")
plt.xlabel("Ngày")
plt.ylabel("Volume")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 6. Boxplot - Phân bố volume
plt.figure(figsize=(6,4))
sns.boxplot(y=df_model['volume'])
plt.title("Phân bố Volume")
plt.ylabel("Volume")
plt.show()

In [ ]:
# 7. Line Plot: Volume Rolling Mean (30 ngày)
plt.figure(figsize=(12,5))
plt.plot(df_model['date'], df_model['volume'].rolling(30).mean(), label='Rolling Mean 30 ngày', color='red')
plt.title("Volume trung bình động (30 ngày)")
plt.xlabel("Ngày")
plt.ylabel("Volume trung bình")
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
# 8. Area Plot: Lượng giao dịch tích lũy (OBV)
plt.figure(figsize=(12,5))
plt.fill_between(df_model['date'], df_model['OBV'], color='purple', alpha=0.5)
plt.title("On-Balance Volume (OBV)")
plt.xlabel("Ngày")
plt.ylabel("OBV")
plt.xticks(rotation=45)
plt.show()

In [ ]:
print(df.columns)

In [ ]:
!pip install pywaffle

In [ ]:
# 9. Waffle Charts - Tỷ lệ số ngày tăng/giảm của giá vàng
from pywaffle import Waffle
import matplotlib.pyplot as plt

total_tiles = 100
total_days = len(df) - 1
data = {
    'Tăng': round((df['Close_Change'] > 0).sum() / total_days * total_tiles),
    'Giảm': round((df['Close_Change'] < 0).sum() / total_days * total_tiles),
    'Không đổi': total_tiles - (
        round((df['Close_Change'] > 0).sum() / total_days * total_tiles) +
        round((df['Close_Change'] < 0).sum() / total_days * total_tiles)
    )
}

fig = plt.figure(
    FigureClass=Waffle,
    rows=10,
    values=data,
    colors=['#00b300', '#e60000', '#999999'],
    title={'label': 'Tỷ lệ ngày Tăng/Giảm Giá Vàng (2015–2025)', 'loc': 'center'},
    legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
    figsize=(8, 5)
)
plt.show()


In [ ]:
df_recent = df_model[df_model['date'] >= pd.Timestamp('2020-01-01')].copy()

total_tiles = 100
total_days = len(df_recent)

data = {
    'Tăng': round((df_recent['Close_Change'] > 0).sum() / total_days * total_tiles),
    'Giảm': round((df_recent['Close_Change'] < 0).sum() / total_days * total_tiles),
    'Không đổi': total_tiles - (
        round((df_recent['Close_Change'] > 0).sum() / total_days * total_tiles) +
        round((df_recent['Close_Change'] < 0).sum() / total_days * total_tiles)
    )
}

fig = plt.figure(
    FigureClass=Waffle,
    rows=10,
    values=data,
    colors=['#00b300', '#e60000', '#999999'],
    title={'label': 'Tỷ lệ ngày Tăng/Giảm Giá Vàng (2020–2025)', 'loc': 'center'},
    legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
    figsize=(8, 5)
)
plt.show()

In [ ]:
# Lọc dữ liệu từ 2024-01-01 trở đi
df_recent = df_model[df_model['date'] >= pd.Timestamp('2024-01-01')].copy()

total_tiles = 100
total_days = len(df_recent)

data = {
    'Tăng': round((df_recent['Close_Change'] > 0).sum() / total_days * total_tiles),
    'Giảm': round((df_recent['Close_Change'] < 0).sum() / total_days * total_tiles),
    'Không đổi': total_tiles - (
        round((df_recent['Close_Change'] > 0).sum() / total_days * total_tiles) +
        round((df_recent['Close_Change'] < 0).sum() / total_days * total_tiles)
    )
}

fig = plt.figure(
    FigureClass=Waffle,
    rows=10,
    values=data,
    colors=['#00b300', '#e60000', '#999999'],
    title={'label': 'Tỷ lệ ngày Tăng/Giảm Giá Vàng (2024–2025)', 'loc': 'center'},
    legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
    figsize=(8, 5)
)
plt.show()


In [ ]:
# 10. Scatter - Khối lượng giao dịch vàng theo ngày
plt.figure(figsize=(15,5))
plt.scatter(df['date'], df['volume'], alpha=0.5, color='green')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.title('Date vs Volume')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# 5. Chatbot Integration

# 6. Model for Prediction

In [ ]:
#Preprocess Data
X = df_model.drop(['date', 'volume'], axis=1)
y = df_model['volume']

#split train test using  Time Series Cross-Validator
tscv = TimeSeriesSplit(n_splits=5)
split = list(tscv.split(X))
train_index, test_index = split[-1]
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#robust scale
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

##1. Linear Regression

In [ ]:
linearR = LinearRegression()
linearR.fit(X_train, y_train)
y_pred = linearR.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

##2. KNN

In [ ]:
knn = KNeighborsRegressor(n_neighbors=3, p=1, weights='distance')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

##3. Random Forest

In [ ]:
randForest = RandomForestRegressor(max_depth=20, max_features=0.8,
                                   min_samples_leaf=2, min_samples_split=2, n_estimators=100,
                                   random_state=42)
randForest.fit(X_train, y_train)
y_pred = randForest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

##4. Polymial Regression

In [ ]:
polyR = make_pipeline(PolynomialFeatures(3), LinearRegression())
polyR.fit(X_train, y_train)
y_pred = polyR.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print('mae:', mae)
print('rmse:', rmse)
print('r2:', r2)

##5. XGBoost

In [ ]:
xgb = XGBRegressor(
    colsample_bytree=1,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=200,
    subsample=1,
    random_state=42
)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print('mae:', mae)
print('rmse:', rmse)
print('r2:', r2)